## N14: sozioökonomische Merkmale gegen Kaufpreis regressieren

In [52]:
df_final.head()

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Niederlassung
0,64,66894.0,88160.31,1.0,43.0,K0310,Bayern
1,54,77644.0,103145.70,1.0,40.0,K1042,Berlin
2,55,44341.0,80565.16,0.0,37.0,K0382,Baden-Württemberg
3,49,67271.0,83949.89,0.0,42.0,K0498,Bayern
4,46,49832.0,93781.58,0.0,41.0,K0552,Hessen


In [53]:
# Einen gefilterten Dataframe erstellen, der nur die Kundendaten enthält 
df_clean = df_final[df_final.index <= 1103] # Alles über dem Index 1103 sind Besucherdaten

In [54]:
df_clean.head()

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Niederlassung
0,64,66894.0,88160.31,1.0,43.0,K0310,Bayern
1,54,77644.0,103145.70,1.0,40.0,K1042,Berlin
2,55,44341.0,80565.16,0.0,37.0,K0382,Baden-Württemberg
3,49,67271.0,83949.89,0.0,42.0,K0498,Bayern
4,46,49832.0,93781.58,0.0,41.0,K0552,Hessen


In [55]:
# Noch einmal prüfen, ob keine NaNs vorhanden sind (Wichtig bei Regression)
df_clean.isna().mean()

Alter            0.0
Einkommen        0.0
Preis            0.0
Geschlecht       0.0
Zeit             0.0
KundeNr          0.0
Niederlassung    0.0
dtype: float64

In [56]:
# Abhängige und unabhängige Variablen definieren
y = df_clean["Preis"]
X = df_clean[["Alter", "Einkommen", "Geschlecht"]]

# Hinzufügen konstanter Term
X = sm.add_constant(X)

# Erstellen und Anpassen des Modells
reg_res = sm.OLS(y, X).fit()

# Anzeigen der Zusammenfassung des Modells
print(reg_res.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     1576.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        22:56:14   Log-Likelihood:                -11797.
No. Observations:                1104   AIC:                         2.360e+04
Df Residuals:                    1100   BIC:                         2.362e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1341.3476   1451.582      0.924      0.3

In [57]:
alpha = reg_res.params["const"]
print("Der Achsenabschnitt beträgt: " + str(round(alpha, 2)))

Der Achsenabschnitt beträgt: 1341.35


In [58]:
beta1 = reg_res.params["Alter"]
print("Eine Erhöhung des Alters um ein Jahr führt zu einer Zunahme des Kaufpreises um: " + str(round(beta1, 2)))

Eine Erhöhung des Alters um ein Jahr führt zu einer Zunahme des Kaufpreises um: 547.94


In [59]:
beta2 = reg_res.params["Einkommen"]
print("Eine Erhöhung des Einkommens um einen Euro führt zu einer Zunahme des Kaufpreises um: " + str(round(beta2, 2)))

Eine Erhöhung des Einkommens um einen Euro führt zu einer Zunahme des Kaufpreises um: 1.0


In [60]:
r_squared = reg_res.rsquared
print("Die Variablen Alter und Einkommen erklären " + str(round(r_squared, 2)*100) + "% der Variabilität beim Preis")

Die Variablen Alter und Einkommen erklären 81.0% der Variabilität beim Preis


Signifikante Prädiktoren: Alle unabhängigen Variablen (Alter, Einkommen) sind signifikante Prädiktoren für den Kaufpreis (p-Werte < 0.05).

## N15: Kaufpreisprognose

Preis = a + ß1 * Alter + ß2 * Einkommen

In [70]:
# Erneut Zielvariable und Prädiktoren definieren
y = df_clean['Preis']
X = df_clean[['Geschlecht', 'Alter', 'Einkommen']]

# Konstanten-Term hinzufügen
X = sm.add_constant(X)

# Modell erstellen und anpassen
model = sm.OLS(y, X).fit()

# Zusammenfassung des Modells anzeigen
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     1576.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        22:57:19   Log-Likelihood:                -11797.
No. Observations:                1104   AIC:                         2.360e+04
Df Residuals:                    1100   BIC:                         2.362e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1341.3476   1451.582      0.924      0.3

In [71]:
# Daten für die Kunden vorbereiten, dessen Kaufpreis wir prognostizieren möchten
neuer_kunde1 = {"const": 1, "ist_maennlich": 1, "Alter": 32, "Einkommen": 30000}
neuer_kunde2 = {"const": 1, "ist_maennlich": 1, "Alter": 51, "Einkommen": 54000}

# DataFrame erstellen
data_to_predict_kunde1 = pd.DataFrame([neuer_kunde1])
data_to_predict_kunde2 = pd.DataFrame([neuer_kunde2])

# Vorhersage machen
predicted_price_kunde1 = model.predict(data_to_predict_kunde1)
predicted_price_kunde2 = model.predict(data_to_predict_kunde2)

print(f"Prognostizierter Kaufpreis für den ersten Kunden: {predicted_price_kunde1[0]:.2f} Euro")
print(f"Prognostizierter Kaufpreis für den zweiten Kunden: {predicted_price_kunde2[0]:.2f} Euro")


Prognostizierter Kaufpreis für den ersten Kunden: 50907.70 Euro
Prognostizierter Kaufpreis für den zweiten Kunden: 85210.14 Euro


## N16: Welche Variable beeinflusst Preis am meisten?

In [72]:
# Zielvariable und Prädiktoren definieren
y = df_clean["Preis"]
X = df_clean[["Geschlecht", "Alter", "Einkommen"]]

# Daten standardisieren (z-Score-Werte berechnen) um Vergleichbarkeit zu schaffen
y_standardized = (y - y.mean()) / y.std()
X_standardized = (X - X.mean()) / X.std()

# Konstante hinzufügen
X_standardized = sm.add_constant(X_standardized)

# Modell erstellen und anpassen
model_standardized = sm.OLS(y_standardized, X_standardized).fit()

# Zusammenfassung des Modells 
print(model_standardized.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     1576.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        22:57:30   Log-Likelihood:                -645.62
No. Observations:                1104   AIC:                             1299.
Df Residuals:                    1100   BIC:                             1319.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.469e-17      0.013  -2.65e-15      1.0

Einkommen hat den größten Einfluss auf den Autopreis, mit einem Koeffizienten von 0.6787, gefolgt von Alter und Geschlecht. Das bestätigt unsere Hypothese, die wir zuvor basierend auf Korrelationskoeffizienten und Scatterplot aufgestellt haben. 
Der Einfluss von Geschlecht ist zwar statistisch signifikant, aber im Vergleich zu Alter und Einkommen relativ gering.

## N17: Wahrscheinlichkeit eines Autokaufs mit logistischer Regression

In [63]:
df_final.isna().mean()

Alter            0.000000
Einkommen        0.000000
Preis            0.446617
Geschlecht       0.000000
Zeit             0.000000
KundeNr          0.000000
Niederlassung    0.000000
dtype: float64

In [64]:
# Wir weisen eine 1 zu, um die Einträge der Kunden zu erfassen, wo ein Preis vorhanden ist
df_final["Kauf"] = np.where(df_final["Preis"].notna(), 1, 0)

# Zielvariable 
y = df_final["Kauf"]
# Regressoren
X = df_final[["Alter", "Einkommen", "Zeit"]]

X = sm.add_constant(X)
logit_model = sm.Logit(y, X).fit()

# Zusammenfassung des Modells
print(logit_model.summary())


Optimization terminated successfully.
         Current function value: 0.580528
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                   Kauf   No. Observations:                 1995
Model:                          Logit   Df Residuals:                     1991
Method:                           MLE   Df Model:                            3
Date:                Wed, 26 Jun 2024   Pseudo R-squ.:                  0.1555
Time:                        22:56:14   Log-Likelihood:                -1158.2
converged:                       True   LL-Null:                       -1371.4
Covariance Type:            nonrobust   LLR p-value:                 3.892e-92
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3090      0.256     -9.026      0.000      -2.810      -1.808
Alter          0.0172      0.

## N18: Wahrscheinlichkeit für Beispiele

In [65]:
# Daten für die neuen Kunden vorbereiten 
neuer_kunde1 = {"Alter": 32,"Einkommen": 30000,"Zeit": 30}
neuer_kunde2 = {"Alter": 51,"Einkommen": 54000,"Zeit": 45}

# DataFrame erstellen für die Vorhersage
X_predict_kunde1 = pd.DataFrame([neuer_kunde1])
X_predict_kunde2 = pd.DataFrame([neuer_kunde2])

# Hinzufügen des konstanten Terms
X_predict_kunde1 = sm.add_constant(X_predict_kunde1, has_constant="add")
X_predict_kunde2 = sm.add_constant(X_predict_kunde2, has_constant="add")

# Vorhersage der Wahrscheinlichkeiten
predicted_probability_kunde1 = logit_model.predict(X_predict_kunde1)
predicted_probability_kunde2 = logit_model.predict(X_predict_kunde2)

print(f"Prognostizierte Wahrscheinlichkeit, dass der erste Kunde ein Modell kauft: {predicted_probability_kunde1[0]:.2f}")
print(f"Prognostizierte Wahrscheinlichkeit, dass der zweite Kunde ein Modell kauft: {predicted_probability_kunde2[0]:.2f}")

Prognostizierte Wahrscheinlichkeit, dass der erste Kunde ein Modell kauft: 0.52
Prognostizierte Wahrscheinlichkeit, dass der zweite Kunde ein Modell kauft: 0.72


## N19: Probleme bei Datenzusammenführung: Operationen zum Zusammenführen und Bereinigen vorstellen

## Datenbereinigung Besucherdatei

In [4]:
df_besucher.head()

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
0,30,"39919,0953348874",0,"29,0",K1814
1,41,"53430,67308587748",0,"27,0",K1544
2,64,"73163,2169267002",0,"32,0",K1945
3,41,"36761,482581315344",0,"44,0",K1825
4,35,"42684,33163178136",1,"33,0",K1144


In [5]:
df_besucher.dtypes

Alter          int64
Einkommen     object
Geschlecht     int64
Zeit          object
KundeNr       object
dtype: object

In [6]:
# Geringer Abstand zwischen Mean und Median (50%) deutet darauf hin, dass es wahrscheinlich nicht viele signifikante Ausreißer gibt 
df_besucher.describe()

,Alter,Geschlecht
count,891.000000,891.000000
mean,49.881033,0.578002
std,11.865889,0.494156
min,30.000000,0.000000
25%,40.000000,0.000000
50%,49.000000,1.000000
75%,60.000000,1.000000
max,70.000000,1.000000


In [7]:
# Erkanntes Problem: Die Zahlen in der Besucherdatei werden von einem Komma und nicht einem Punkt getrennt 
# Außerdem liegen die Spalten Zeit und Einkommen im String-Format vor
# mit replace führen wir den Austausch von , zu . durch und speichern dann als float 

df_besucher['Zeit'] = df_besucher['Zeit'].str.replace(',', '.').astype(float)
df_besucher['Einkommen'] = df_besucher['Einkommen'].str.replace(',', '.').astype(float)


In [8]:
# Die Änderungen wurden nun übernommen
df_besucher.head()

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
0,30,39919.095335,0,29.0,K1814
1,41,53430.673086,0,27.0,K1544
2,64,73163.216927,0,32.0,K1945
3,41,36761.482581,0,44.0,K1825
4,35,42684.331632,1,33.0,K1144


In [9]:
# Zsore für die Suche nach Ausreißern bestimmen

df_besucher["Alter_zscore"] = zscore(df_besucher.Alter)
df_besucher["Einkommen_zscore"] = zscore(df_besucher.Einkommen)
df_besucher["Zeit_zscore"] = zscore(df_besucher.Zeit)

In [10]:
# Mit der query Methode alle Zeilen nach Zscore > / < 3 Standardabweichungen filtern
df_besucher.query("Alter_zscore > 3")

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Zeit_zscore


In [11]:
df_besucher.query("Alter_zscore < -3")

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Zeit_zscore


In [12]:
df_besucher.query("Einkommen_zscore > 3")

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Zeit_zscore
44,69,108569.087021,1,47.0,K1467,1.61216,3.045514,1.130167


In [13]:
df_besucher.query("Einkommen_zscore < -3")

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Zeit_zscore
265,31,10802.033307,1,25.0,K1561,-1.592096,-3.006193,-0.970889


In [14]:
df_besucher.query("Zeit_zscore > 3")

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Zeit_zscore
603,62,84204.706728,1,68.0,K1526,1.021902,1.537377,3.135722


In [15]:
df_besucher.query("Zeit_zscore < -3")

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Zeit_zscore
158,31,38298.533652,1,3.0,K1623,-1.592096,-1.30418,-3.071946


In [16]:
# Zeigt die fehlenden Werte je Zeile in Prozent an
df_besucher.isna().mean()

Alter               0.0
Einkommen           0.0
Geschlecht          0.0
Zeit                0.0
KundeNr             0.0
Alter_zscore        0.0
Einkommen_zscore    0.0
Zeit_zscore         0.0
dtype: float64

In [17]:
# Ausreißer und NaNs mit Median ersetzen (Median um unanfälliger gegenüber Ausreißern zu sein als bei Durchschnitt)

ersetz_eink = [44, 265] # Zu ersetzende Indexpositionen  
ersetz_zeit = [603, 158]

# Mit loc Funktion Index und Spalte angeben und dann mit Median ersetzen
df_besucher.loc[ersetz_eink, 'Einkommen'] = df_besucher.Einkommen.median()
df_besucher.loc[ersetz_zeit, 'Zeit'] = df_besucher.Zeit.median()
#df_besucher.Geschlecht.fillna(df_besucher.Geschlecht.median(), inplace=True)

In [18]:
# Zscore Spalten werden nicht mehr benötigt und können gelöscht werden
df_besucher.drop(columns=["Alter_zscore", "Einkommen_zscore", "Zeit_zscore"], inplace=True)

## Datenbereinigung Kundendatei

In [19]:
df_kunden.head()

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1.0,43.0,K0310
1,54,77644.0,103145.70,1.0,40.0,K1042
2,55,44341.0,80565.16,0.0,37.0,K0382
3,49,67271.0,83949.89,0.0,42.0,K0498
4,46,49832.0,93781.58,0.0,41.0,K0552


In [20]:
df_kunden.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht    float64
Zeit          float64
KundeNr        object
dtype: object

In [21]:
df_kunden.describe()

,Alter,Einkommen,Preis,Geschlecht,Zeit
count,1104.000000,1.104000e+03,1104.000000,1099.000000,1104.000000
mean,50.481884,9.644748e+05,88975.198714,0.637853,44.375906
std,12.225059,3.009470e+07,24464.067781,0.480840,10.848040
min,30.000000,-6.284037e+04,21471.650000,0.000000,12.000000
25%,40.000000,4.663850e+04,70609.422500,0.000000,37.000000
50%,51.000000,5.901300e+04,88931.455000,1.000000,44.000000
75%,61.000000,7.029800e+04,107093.375000,1.000000,52.000000
max,120.000000,1.000000e+09,165482.310000,1.000000,80.000000


In [22]:
# Zscore für Ausreißer

df_kunden["Alter_zscore"] = zscore(df_kunden.Alter)
df_kunden["Einkommen_zscore"] = zscore(df_kunden.Einkommen)
df_kunden["Preis_zscore"] = zscore(df_kunden.Preis)
df_kunden["Zeit_zscore"] = zscore(df_kunden.Zeit)

In [23]:
df_kunden.query("Alter_zscore > 3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore
142,100,88749.0,119134.23,0.0,64.0,K0494,4.052378,-0.029112,1.233348,1.809819
711,120,61857.0,104214.07,1.0,52.0,K0039,5.689103,-0.030006,0.623191,0.703127


In [24]:
df_kunden.query("Alter_zscore < -3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore


In [25]:
df_kunden.query("Alter < 0")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore


In [26]:
df_kunden.query("Einkommen_zscore > 3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore
408,68,1.000000e+09,87209.72,0.0,52.0,K0701,1.433617,33.211439,-0.072199,0.703127


In [27]:
df_kunden.query("Einkommen_zscore < -3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore


In [28]:
df_kunden.query("Einkommen < 0")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore
1082,59,-62840.374523,92837.59,0.0,46.0,K1083,0.697091,-0.034152,0.157952,0.149781


In [29]:
df_kunden.query("Preis_zscore > 3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore
834,68,107689.0,165482.31,1.0,60.0,K0415,1.433617,-0.028483,3.128743,1.440922


In [30]:
df_kunden.query("Preis_zscore < -3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore


In [31]:
df_kunden.query("Preis < 0")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore


In [32]:
df_kunden.query("Zeit_zscore > 3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore
313,62,96569.0,140745.65,0.0,80.0,K0249,0.9426,-0.028852,2.117142,3.285408


In [33]:
df_kunden.query("Zeit_zscore < -3")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore


In [34]:
df_kunden.query("Zeit < 0")

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Alter_zscore,Einkommen_zscore,Preis_zscore,Zeit_zscore


In [35]:
# Zeigt die fehlenden Werte je Zeile in Prozent an
df_kunden.isna().mean()

Alter               0.000000
Einkommen           0.000000
Preis               0.000000
Geschlecht          0.004529
Zeit                0.000000
KundeNr             0.000000
Alter_zscore        0.000000
Einkommen_zscore    0.000000
Preis_zscore        0.000000
Zeit_zscore         0.000000
dtype: float64

In [36]:
# Ausreißer und NaNs durch Median ersetzen - gleiches Vorgehen wie davor

ersetz_alter_2 = [142, 711]
ersetz_einkommen_2 = [408, 1082]
ersetzZeit_2 = 313
ersetzPreis_2 = 834

df_kunden.loc[ersetz_alter_2, 'Alter'] = df_kunden.Alter.median()
df_kunden.loc[ersetz_einkommen_2, 'Einkommen'] = df_kunden.Einkommen.median()
df_kunden.loc[ersetzZeit_2, 'Zeit'] = df_kunden.Zeit.median()
df_kunden.loc[ersetzPreis_2, 'Preis'] = df_kunden.Preis.median()

df_kunden.Geschlecht.fillna(df_kunden.Geschlecht.median(), inplace=True)

In [37]:
df_kunden.Einkommen.median()

59013.0

In [38]:
# Zscore Spalten wieder entfernen
df_kunden.drop(columns=["Alter_zscore", "Einkommen_zscore", "Preis_zscore", "Zeit_zscore"], inplace=True)

In [39]:
df_kunden.head()

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1.0,43.0,K0310
1,54,77644.0,103145.70,1.0,40.0,K1042
2,55,44341.0,80565.16,0.0,37.0,K0382
3,49,67271.0,83949.89,0.0,42.0,K0498
4,46,49832.0,93781.58,0.0,41.0,K0552


## Datenbereinigung Geodatei

In [40]:
txt_geo.head()

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,NRW
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern


In [41]:
# Die Datei enthält zwei Spalten, jeweils Strings
txt_geo.dtypes

KundeNr          object
Niederlassung    object
dtype: object

In [42]:
# Testen, welche Bundesländer es gibt und welche Ungereimtheiten auftreten

print(txt_geo.Niederlassung.unique())
print("Einzigartige Einträge: " + str(txt_geo.Niederlassung.nunique()))

['Sachsen' 'NRW' 'Hessen' 'Bayern' 'Baden-Württemberg'
 'Nordrhein-Westfalen' 'Berlin' 'Niedersachsen' 'Brandenburg' 'Thüringen'
 'Hamburg' 'Düsseldorf' 'Berlin-Charlottenburg' 'Berlin-Mitte' 'BERLIN']
Einzigartige Einträge: 15


In [43]:
# Einheitliches Format erzeugen 

txt_geo.Niederlassung.replace({'NRW': 'Nordrhein-Westfalen',
                            'Berlin-Charlottenburg': 'Berlin',
                            'Berlin-Mitte': 'Berlin',
                            'BERLIN': 'Berlin'}, inplace=True)

In [44]:
# Testen, ob die Korrekturen durchgeführt wurden
# Zuvor gab es 14 einzigartige Bundesländer, jetzt noch 11. Das ist korrekt. 
print("Einzigartige Einträge: " + str(txt_geo.Niederlassung.nunique()))

Einzigartige Einträge: 11


# Daten in einen Dataframe zusammenfügen

In [45]:
# Merge der Datensätze Kunden und Besucher
df1 = pd.merge(df_kunden, df_besucher, on=['KundeNr', 'Alter', 'Einkommen', 'Geschlecht', 'Zeit'], how='outer') #Methode outer, da ungleich lang und alle Daten behalten werden sollen
# Für df_final ebenfalls die Standortdatei txt_geo hinzufügen
df_final = pd.merge(df1, txt_geo, on='KundeNr', how='inner') # Methode inner, da nur die Zeilen, wo KundenNr übereinstimmt behalten werden sollen 

In [46]:
df1.head()

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1.0,43.0,K0310
1,54,77644.0,103145.70,1.0,40.0,K1042
2,55,44341.0,80565.16,0.0,37.0,K0382
3,49,67271.0,83949.89,0.0,42.0,K0498
4,46,49832.0,93781.58,0.0,41.0,K0552


In [47]:
# Anhand beispielhafter KundenNr aus Besucher-Datensatz prüfen, ob die Daten richtig übertragen wurden
df_besucher.query("KundeNr == 'K1945'") # Da es sich um String handelt, wird die Kundennr in weitere Anführungszeichen genommen 

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
2,64,73163.216927,0,32.0,K1945


In [48]:
# Wurde übernommen 
df1.query("KundeNr == 'K1945'") 

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
1106,64,73163.216927,NaN,0.0,32.0,K1945


In [49]:
# Übersicht finaler Dataframe
df_final.head()

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Niederlassung
0,64,66894.0,88160.31,1.0,43.0,K0310,Bayern
1,54,77644.0,103145.70,1.0,40.0,K1042,Berlin
2,55,44341.0,80565.16,0.0,37.0,K0382,Baden-Württemberg
3,49,67271.0,83949.89,0.0,42.0,K0498,Bayern
4,46,49832.0,93781.58,0.0,41.0,K0552,Hessen


In [50]:
# Überprüfen ob die richtigen Datenformate vorliegen
# Nur die Spalten KundeNr und Niederlassung sind strings, was korrekt ist
df_final.dtypes

Alter              int64
Einkommen        float64
Preis            float64
Geschlecht       float64
Zeit             float64
KundeNr           object
Niederlassung     object
dtype: object

In [51]:
# Nach NaNs gucken - Wie erwartet fehlen nur in Preisspalte einige Einträge (aus Besucherdatei)
df_final.isna().mean()

Alter            0.000000
Einkommen        0.000000
Preis            0.446617
Geschlecht       0.000000
Zeit             0.000000
KundeNr          0.000000
Niederlassung    0.000000
dtype: float64

## N20: Zwei Vorschläge für zusätzliche Datenerhebungen 
#### vllt. auch 3 ;)

1. Beruf der Kunden
2. Lief zu dem Zeitpunkt des Verkaufs eine Rabattaktion? (Wenn ja Prozentsatz angeben)
3. Investitionen in Vermarktung 